# LastFM Recommender Engine: Data Cleaning
June [x] 2018

In [1]:
import numpy as np
import pandas as pd

In [2]:
# sets output to three decimals
pd.set_option('display.float_format',lambda x: '%.3f' %x)

In [3]:
# six files provided
artists = pd.read_csv('../data/artists.dat', sep='\t',usecols=['id','name'])
plays = pd.read_csv('../data/user_artists.dat', sep='\t')
tags = pd.read_csv('../data/tags.dat', sep='\t',encoding='latin-1')
friends = pd.read_csv('../data/user_friends.dat', sep='\t') # we will not use the social network in our recommender
utat = pd.read_csv('../data/user_taggedartists-timestamps.dat', sep="\t")
uta = pd.read_csv('../data/user_taggedartists.dat', sep='\t')

### File statistics

In [4]:
# Explore specs of each file provided.  We will only use artists and plays in the basic system.
csv_list = [artists, plays, tags, friends, utat,uta]
csv_names = ['artists', 'plays', 'tags', 'friends', 'user_taggedartists-timestamps','user_taggedartists']
for i in range(len(csv_list)):
    name = csv_names[i]
    shape = csv_list[i].shape
    columns = csv_list[i].columns
    unique = csv_list[i].nunique()
    print("{}\n{}\n{}\n{}\n".format(name, shape, columns,unique))

artists
(17632, 2)
Index(['id', 'name'], dtype='object')
id      17632
name    17632
dtype: int64

plays
(92834, 3)
Index(['userID', 'artistID', 'weight'], dtype='object')
userID       1892
artistID    17632
weight       5436
dtype: int64

tags
(11946, 2)
Index(['tagID', 'tagValue'], dtype='object')
tagID       11946
tagValue    11946
dtype: int64

friends
(25434, 2)
Index(['userID', 'friendID'], dtype='object')
userID      1892
friendID    1892
dtype: int64

user_taggedartists-timestamps
(186479, 4)
Index(['userID', 'artistID', 'tagID', 'timestamp'], dtype='object')
userID        1892
artistID     12523
tagID         9749
timestamp     3549
dtype: int64

user_taggedartists
(186479, 6)
Index(['userID', 'artistID', 'tagID', 'day', 'month', 'year'], dtype='object')
userID       1892
artistID    12523
tagID        9749
day             4
month          12
year           10
dtype: int64



### Prepare artist plays matrix

In [5]:
artist_plays = pd.merge(artists, plays,how='left',left_on='id',right_on='artistID')
artist_plays = artist_plays.drop(columns=['id']) # drop duplicate columns

In [6]:
# shape of plays is 92,834
artist_plays.shape

(92834, 4)

In [7]:
artist_plays.head()

,name,userID,artistID,weight
0,MALICE MIZER,34,1,212
1,MALICE MIZER,274,1,483
2,MALICE MIZER,785,1,76
3,Diary of Dreams,135,2,1021
4,Diary of Dreams,257,2,152


In [8]:
artist_plays[artist_plays['name']=='MALICE MIZER']

,name,userID,artistID,weight
0,MALICE MIZER,34,1,212
1,MALICE MIZER,274,1,483
2,MALICE MIZER,785,1,76


In [9]:
artist_plays[artist_plays['name']=='Britney Spears']['weight'].sum()

2393140

In [10]:
# Average plays by user
artist_plays['weight'].describe()

count    92834.000
mean       745.244
std       3751.322
min          1.000
25%        107.000
50%        260.000
75%        614.000
max     352698.000
Name: weight, dtype: float64

In [11]:
total_plays = artist_plays[['name','weight']].groupby('name').sum()

print(total_plays['weight'].quantile(np.arange(.9,1.,.01)))

0.900    4645.400
0.910    5350.680
0.920    6193.000
0.930    7320.640
0.940    8685.280
0.950   10693.400
0.960   14257.800
0.970   18969.740
0.980   30137.940
0.990   60096.010
Name: weight, dtype: float64


In [12]:
# popularity threshold to work with most popular artists
pop_thresh = 20000
popular_artists = total_plays[total_plays['weight']>pop_thresh]
print("Removing artists with total plays below {}".format(pop_thresh))
ap2 = artist_plays[artist_plays['name'].isin(popular_artists.index)] 
print("{} popular artists".format(popular_artists.shape))
print("Shape before: {} Shape after: {}".format(artist_plays.shape,ap2.shape))

Removing artists with total plays below 20000
(507, 1) popular artists
Shape before: (92834, 4) Shape after: (44301, 4)


In [13]:
# those below popularity threshold have been removed
assert artist_plays[artist_plays['name']=='Tutu Jones']['weight'].sum()>0
assert ap2[ap2['name']=='Tutu Jones']['weight'].sum()==0

In [14]:
artist_play = ap2

In [15]:
artist_play.shape

(44301, 4)

In [16]:
artist_play.nunique()

name         507
userID      1861
artistID     507
weight      4803
dtype: int64

In [17]:
# Each artist played a minimum of times based on popularity threshold above
artist_play.groupby('name').sum().sort_values('weight').head()

,userID,artistID,weight
name,,,
Roxette,45654,8820,20046
Thrice,15392,35302,20123
De/Vision,28695,5049,20203
Van Halen,54294,126738,20205
Recoil,26897,33952,20209


In [18]:
print(artist_plays.name.nunique(), artist_play.index.nunique())

17632 44301


In [19]:
artist_play.head()

,name,userID,artistID,weight
32,Marilyn Manson,23,7,212
33,Marilyn Manson,59,7,1410
34,Marilyn Manson,85,7,185
35,Marilyn Manson,124,7,102
36,Marilyn Manson,127,7,419


In [20]:
artist_play.name.nunique()

507

In [21]:
artist_play2 = artist_play.pivot(index='name',columns='userID',values='weight').fillna(0)

In [22]:
artist_play2.head()

userID,2,3,4,5,6,7,8,9,10,11,...,2090,2091,2092,2093,2094,2095,2096,2097,2099,2100
name,,,,,,,,,,,,,,,,,,,,,
3 Doors Down,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,514.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
30 Seconds to Mars,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3OH!3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1117.000,...,0.000,0.000,0.000,0.000,1528.000,0.000,0.000,0.000,0.000,0.000
50 Cent,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A Day to Remember,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [23]:
artist_play2.shape

(507, 1861)

### Prepare artist tag matrix

In [24]:
user_tagged = pd.merge(utat,uta,how='outer',on=['userID','tagID','artistID'])
assert len(user_tagged) == len(utat) == len(uta)

In [25]:
# left join as assuming we do not needs tags that have not been used by users
user_tags = pd.merge(user_tagged,tags,how='left',on='tagID')

In [26]:
user_tags2 = pd.merge(user_tags,artists,how='left',left_on='artistID',right_on='id')
user_tags3 = user_tags2.drop(columns='id')

In [27]:
user_tags3.head()

,userID,artistID,tagID,timestamp,day,month,year,tagValue,name
0,2,52,13,1238536800000,1,4,2009,chillout,Morcheeba
1,2,52,15,1238536800000,1,4,2009,downtempo,Morcheeba
2,2,52,18,1238536800000,1,4,2009,electronic,Morcheeba
3,2,52,21,1238536800000,1,4,2009,trip-hop,Morcheeba
4,2,52,41,1238536800000,1,4,2009,female vovalists,Morcheeba


In [28]:
ut3 = user_tags3[['name','tagValue']]
ut3a = ut3.groupby('tagValue').count().sort_values('name',ascending=False)

In [29]:
ut3a.head()

,name
tagValue,
rock,7459
pop,5401
alternative,5223
electronic,4616
indie,4422


In [30]:
ut3b = ut3a.quantile(np.arange(.9,1,.01))

In [31]:
ut3b

,name
0.900,17.000
0.910,20.000
0.920,24.000
0.930,28.000
0.940,33.000
0.950,40.000
0.960,49.000
0.970,67.000
0.980,122.000
0.990,322.600


In [32]:
# popularity threshold for most popular tags
tag_thresh = 40
popular_tags = ut3a[ut3a['name']>tag_thresh]
print("Removing tags with total frequency below {}".format(tag_thresh))
ut3c = user_tags3[user_tags3['tagValue'].isin(popular_tags.index)] 
print("{} popular tags".format(popular_tags.shape))
print("Shape before: {} Shape after: {}".format(user_tags3.shape,ut3c.shape))

Removing tags with total frequency below 40
(478, 1) popular tags
Shape before: (186479, 9) Shape after: (150137, 9)


In [33]:
ut3c.head()

,userID,artistID,tagID,timestamp,day,month,year,tagValue,name
0,2,52,13,1238536800000,1,4,2009,chillout,Morcheeba
1,2,52,15,1238536800000,1,4,2009,downtempo,Morcheeba
2,2,52,18,1238536800000,1,4,2009,electronic,Morcheeba
3,2,52,21,1238536800000,1,4,2009,trip-hop,Morcheeba
5,2,63,13,1238536800000,1,4,2009,chillout,Enigma


In [34]:
# popularity threshold to work with most popular artists
ut3d = ut3c[ut3c['name'].isin(popular_artists.index)] 
print("{} popular artists".format(popular_artists.shape))
print("Shape before: {} Shape after: {}".format(ut3c.shape,ut3d.shape))

(507, 1) popular artists
Shape before: (150137, 9) Shape after: (52644, 9)


In [35]:
# create sparse matrix of artists with tags
ut4 = ut3d[['name','tagValue']].drop_duplicates()
ut5 = pd.concat([ut4.drop('tagValue', 1), pd.get_dummies(ut4.tagValue).mul(1)], axis=1)
ut6 = ut5.groupby('name').sum()

In [36]:
ut6

,00s,1008,10s,1980s,1981,1982,1991,1992,1995,1996,...,usa,viking metal,visual kei,vocal,vocal trance,want to see live,weekly top tracks,world,world music,worship
name,,,,,,,,,,,,,,,,,,,,,
3 Doors Down,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30 Seconds to Mars,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3OH!3,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
50 Cent,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Day to Remember,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Fine Frenzy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Perfect Circle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Skylit Drive,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ABBA,1,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
print(tags.tagValue.shape,ut6.columns.shape)

(11946,) (457,)


In [38]:
ut6.shape

(505, 457)

### Convert files into serialized and readable formats

In [39]:
print("Dataframe Shapes")
print("Artist Plays: {}".format(artist_play2.shape))
print("User Tags: {}".format(ut6.shape))
print("Friends: {}".format(friends.shape))

Dataframe Shapes
Artist Plays: (507, 1861)
User Tags: (505, 457)
Friends: (25434, 2)


In [40]:
# these files must be kept separate, as each userID/artistID pair features multiple tagID
# and each userID is associated with multiple friendID
artist_play2.to_pickle('../data/recsys_plays.pkl')
ut6.to_pickle('../data/recsys_tags.pkl')
friends.to_pickle('../data/recsys_friends.pkl')

In [41]:
# aggregate data for FusionAI@Lucidworks exercise
artist_play2.to_csv('../data/recsys_plays.csv', index=False)
ut6.to_csv('../data/recsys_tags.csv', index=False)

Below: Experiment with different data structures, such as json/dictionary, for Fusion ingestion

In [51]:
n = 20 #len(user_tags3)

ad = {}
for i in range(n):
    name = user_tags3['name'].iloc[i]
    tagValue = user_tags3['tagValue'].iloc[i]
    if name in ad:
        ad[name].append(tagValue)
    else:
        ad[name] = [tagValue]

In [52]:
ad

{'Café Del Mar': ['chillout',
  'ambient',
  'downtempo',
  'electronic',
  'lounge',
  'trip-hop',
  'acid jazz',
  'atmospheric'],
 'Enigma': ['chillout', 'ambient', 'new age', 'gregorian chant'],
 'Ministry of Sound': ['chillout', 'downtempo', 'lounge'],
 'Morcheeba': ['chillout',
  'downtempo',
  'electronic',
  'trip-hop',
  'female vovalists']}

In [53]:
user_tags4 = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ad.items() ]))

In [54]:
user_tags4

,Café Del Mar,Enigma,Ministry of Sound,Morcheeba
0,chillout,chillout,chillout,chillout
1,ambient,ambient,downtempo,downtempo
2,downtempo,new age,lounge,electronic
3,electronic,gregorian chant,NaN,trip-hop
4,lounge,NaN,NaN,female vovalists
5,trip-hop,NaN,NaN,NaN
6,acid jazz,NaN,NaN,NaN
7,atmospheric,NaN,NaN,NaN


In [55]:
# user_tags4 = pd.DataFrame.from_dict(ad)

In [56]:
user_tags3.iloc[1]

userID                   2
artistID                52
tagID                   15
timestamp    1238536800000
day                      1
month                    4
year                  2009
tagValue         downtempo
name             Morcheeba
Name: 1, dtype: object